In [ ]:
import geopandas as gpd
import pandas as pd
import re
import MyData
from IPython.display import clear_output

def str_to_point(_):
    thelong = _.Point.map(lambda x: [float(y) for y in re.findall(r'[\d]+[.]?[\d]*',str(x))][0])
    thelat = _.Point.map(lambda x: [float(y) for y in re.findall(r'[\d]+[.]?[\d]*',str(x))][1])
    return gpd.points_from_xy(thelong, thelat)

cashed_metric = pd.read_csv('cashed_llp.csv',index_col='UID')
df = pd.read_csv('./NOAA_CHINA_1949-2019_located_20200803.csv',encoding='utf-8')
df['Point'] = gpd.points_from_xy(df.LONGITUDE, df.LATITUDE)
gdf = gpd.read_file('./geography/分县.shp')
gdf['Point'] = gdf.centroid
gdf.Point = str_to_point(gdf)

In [ ]:
import matplotlib.pyplot as plt
def split_polygon(plg):
    if isinstance(plg,gpd.geodataframe.GeoDataFrame):
        print('Is Polygon')
        X = []
        Y = []
        for plg in plg.geometry:
            #point_list = plg.wkt.replace('POLYGON','').strip('(|)| ').split(', ')
            point_list = re.findall(r'[\d]+[.]*[\d]* [\d]+[.]*[\d]*', plg.wkt)
            X.extend([float(x.split(' ')[0]) for x in point_list])
            Y.extend([float(y.split(' ')[1]) for y in point_list])

    elif isinstance(plg,shapely.geometry.polygon.Polygon):
        print('Not Polygon')
        point_list = re.findall(r'[\d]+[.]*[\d]* [\d]+[.]*[\d]*', plg.wkt)
        X = [float(x.split(' ')[0]) for x in point_list]
        Y = [float(y.split(' ')[1]) for y in point_list]

    else:
        raise ValueError('Split Polygon error')
    return X,Y

%matplotlib inline

def plot_kw(keywords: list, gdf):
    ax = gdf.plot(figsize=(10,10),color='grey')
    iconcerns = []
    for kw in keywords:
        iconcerns.append(gdf[gdf['分县连接成'].str.contains(kw) == True])
    iconcern = pd.concat(iconcerns)

    #point_iconcern = gpd.GeoSeries(shapely.geometry.Point(113.8565, 22.68435),crs="EPSG:4326")
    #point_iconcern.plot(ax=ax,color='black')

    iconcern.plot(ax=ax,color='green',edgecolor='red',alpha=0.9)
    x,y = split_polygon(iconcern)
    x_max = max(x)
    x_min = min(x)
    x_dif = (x_max - x_min)*0.5
    y_max = max(y)
    y_min = min(y)
    y_dif = y_max - y_min
    plt.xlim(x_min-x_dif,x_max+x_dif)
    plt.ylim(y_min-y_dif,y_max+y_dif)
    return ax, iconcern

%matplotlib inline
kw = ['广东']
ax,iconcern = plot_kw(kw, gdf)
gpd.GeoSeries(iconcern.Point).plot(ax=ax,color='Yellow')

In [ ]:
def full_data_on_date(df, mygdf, thedate, datacol, max_distance, min_n, max_n):
    datadf = df[df.DATE == thedate]
    datadf = MyData.format_df_cols(datadf.NAME, datadf.Point, thevalues=datadf[datacol])
    mymd = MyData.MyData(datadf, mygdf, cashed_metric = cashed_metric, instance_name='%s,%s' %(datacol,thedate))
    newdata, cost_time = mymd.interpolate(max_distance, min_n, max_n)
    newdata.name = datacol
    dataresult = pd.concat([newdata,mymd.df.status],axis=1)
    dataresult['date'] = thedate
    dataresult['round'] = mymd.round
    dataresult['datasize'] = '%d/%d' %(mymd.datasize, mymd.total_datasize)
    dataresult['cost_time'] = cost_time
    return dataresult,mymd

#mygdf = MyData.format_df_cols(gdf.NAME99, gdf.Point)
mygdf = MyData.format_df_cols(iconcern.分县连接成, iconcern.Point)

datacols = ['TEMP','MAX','MIN','WDSP','MXSPD','PRCP','SNDP','GUST','DEWP','STP']
clean_count = 0
'''year_with_abundant_data = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]'''
year_with_abundant_data = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

for datacol in datacols:
    current_threequat = df[datacol].describe()['75%']
    mydf = df[(df[datacol].map(lambda thenum: current_threequat<=thenum and [set(x.replace('.','')) for x in re.findall(r'[\d]+.?[\d]*',str(thenum))] == [set('9')]))!=True]
    for year in year_with_abundant_data:
        if datacol in ['TEMP','MAX','MIN','WDSP','MXSPD','PRCP']:
            continue
        dates = mydf[mydf.YEAR == year].DATE.unique()
        year_dfs = []
        for date in dates:
            date_df,thismd = full_data_on_date(df, mygdf, date, datacol, 100, 3, 20)
            year_dfs.append(date_df)
            clean_count += 1
            if clean_count >= 3:
                clear_output(wait=True)
                clean_count = 0
        if year_dfs == []:
            continue
        year_df = pd.concat(year_dfs)
        year_df.to_csv('./data_result/ineed_%s_%s.csv' %(datacol, year),encoding='utf-8')

In [ ]:
thismd.llp.mymetric.to_csv('cashed_llp.csv',encoding='utf-8')